In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub
from google.colab import files

# Download latest version
path = kagglehub.dataset_download("rohanrao/air-quality-data-in-india")

print("Path to dataset files:", path)

In [ ]:
!pip install pmdarima

In [ ]:
# Library imports and setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings
warnings.filterwarnings('ignore')

# Configure plot settings
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Data loading and preprocessing
def load_and_preprocess_data():
    # Load data
    data = pd.read_csv('/kaggle/input/air-quality-data-in-india/city_hour.csv', 
                      parse_dates=['Datetime'], 
                      index_col='Datetime')
    
    # Select data for Delhi and PM2.5
    city_data = data[data['City'] == 'Delhi'][['PM2.5']].dropna()
    
    # Convert to daily data
    daily_data = city_data.resample('D').mean().dropna()
    
    return daily_data

# Call data loading function
ts_data = load_and_preprocess_data()

# Data inspection
print("First 5 entries:")
print(ts_data.head())
print("\nStatistical description:")
print(ts_data.describe())
print("\nNumber of missing values:", ts_data.isna().sum())

# Time series visualization
plt.figure(figsize=(14, 6))
plt.plot(ts_data, label='PM2.5 (Delhi, daily)')
plt.title('PM2.5 Time Series for Delhi')
plt.xlabel('Date')
plt.ylabel('PM2.5 Concentration')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Data cleaning and anomaly removal
def clean_data(data):
    # Anomaly detection
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    anomalies = iso_forest.fit_predict(data.values.reshape(-1, 1))
    
    # Remove anomalies
    cleaned_data = data[anomalies == 1]
    
    # Fill missing values
    cleaned_data = cleaned_data.reindex(data.index)
    cleaned_data = cleaned_data.interpolate(method='time').fillna(method='ffill').fillna(method='bfill')
    
    return cleaned_data

# Clean the data
cleaned_ts = clean_data(ts_data)

# Verify cleaning results
print("\nData size before cleaning:", len(ts_data))
print("Data size after cleaning:", len(cleaned_ts))
print("Missing values after cleaning:", cleaned_ts.isna().sum())

# Visualize cleaned data
plt.figure(figsize=(14, 6))
plt.plot(cleaned_ts, label='Cleaned PM2.5 Series', color='green')
plt.title('Time Series after Cleaning')
plt.xlabel('Date')
plt.ylabel('PM2.5 Concentration')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Time series analysis
def analyze_time_series(data):
    # Stationarity test
    result = adfuller(data)
    print("\nAugmented Dickey-Fuller Test Results:")
    print(f"p-value: {result[1]:.4f}")
    if result[1] > 0.05:
        print("Series is non-stationary - differencing is required")
    else:
        print("Series is stationary")
    
    # Series decomposition
    decomposition = seasonal_decompose(data, model='additive', period=30)
    
    # Visualize decomposition
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 10))
    decomposition.observed.plot(ax=ax1, title='Original Series')
    decomposition.trend.plot(ax=ax2, title='Trend')
    decomposition.seasonal.plot(ax=ax3, title='Seasonality')
    decomposition.resid.plot(ax=ax4, title='Residuals')
    plt.tight_layout()
    plt.show()

# Perform analysis
analyze_time_series(cleaned_ts['PM2.5'])

In [ ]:
# Splitting into training and test sets
train_size = int(len(cleaned_ts) * 0.8)
train = cleaned_ts.iloc[:train_size]
test = cleaned_ts.iloc[train_size:]

print(f"\nTraining set size: {len(train)}")
print(f"Test set size: {len(test)}")

# Visualize the split
plt.figure(figsize=(14, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Test Data')
plt.title('Training and Test Set Split')
plt.xlabel('Date')
plt.ylabel('PM2.5 Concentration')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecasting using ARIMA
def arima_forecast(train_data, test_data):
    try:
        print("\nFitting ARIMA model...")
        model = auto_arima(train_data, seasonal=True, m=12,
                           suppress_warnings=True,
                           stepwise=True, trace=True)
        
        print("\nObtained model parameters:")
        print(model.summary())
        
        # Forecast
        forecast = model.predict(n_periods=len(test_data))
        forecast = pd.Series(forecast, index=test_data.index)
        
        # Fill any potential NaN values
        forecast = forecast.fillna(method='ffill').fillna(method='bfill')
        
        return forecast
    
    except Exception as e:
        print(f"\nError building ARIMA model: {str(e)}")
        return None


# Perform ARIMA forecast
arima_pred = arima_forecast(train['PM2.5'], test['PM2.5'])

# Visualize ARIMA results
if arima_pred is not None:
    plt.figure(figsize=(14, 6))
    plt.plot(train['PM2.5'], label='Training Data')
    plt.plot(test['PM2.5'], label='Actual Values')
    plt.plot(arima_pred, label='ARIMA Forecast', linestyle='--')
    plt.title('Forecasting using ARIMA')
    plt.xlabel('Date')
    plt.ylabel('PM2.5 Concentration')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Forecasting using Holt-Winters
def holt_winters_forecast(train_data, test_data):
    try:
        print("\nTraining Holt-Winters model...")
        model = ExponentialSmoothing(train_data,
                                     seasonal='add',
                                     trend='add',
                                     seasonal_periods=30)
        model_fit = model.fit()
        
        # Forecast
        forecast = model_fit.forecast(len(test_data))
        forecast = pd.Series(forecast, index=test_data.index)
        
        return forecast
    
    except Exception as e:
        print(f"\nError building Holt-Winters model: {str(e)}")
        return None

# Perform Holt-Winters forecast
hw_pred = holt_winters_forecast(train['PM2.5'], test['PM2.5'])

# Visualize Holt-Winters results
if hw_pred is not None:
    plt.figure(figsize=(14, 6))
    plt.plot(train['PM2.5'], label='Training Data')
    plt.plot(test['PM2.5'], label='Actual Values')
    plt.plot(hw_pred, label='Holt-Winters Forecast', linestyle='--')
    plt.title('Forecasting using Holt-Winters')
    plt.xlabel('Date')
    plt.ylabel('PM2.5 Concentration')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Forecasting using LSTM
def lstm_forecast(train_data, test_data, n_steps=30, epochs=50):
    try:
        print("\nPreparing data for LSTM...")
        # Normalize data
        scaler = MinMaxScaler()
        train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
        test_scaled = scaler.transform(test_data.values.reshape(-1, 1))
        
        # Create sequences
        def create_sequences(data, seq_length):
            X, y = [], []
            for i in range(len(data)-seq_length):
                X.append(data[i:i+seq_length])
                y.append(data[i+seq_length])
            return np.array(X), np.array(y)
        
        X_train, y_train = create_sequences(train_scaled, n_steps)
        X_test, y_test = create_sequences(test_scaled, n_steps)
         
        # Build model
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(n_steps, 1)),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        
        # Train model
        print("\nTraining LSTM model...")
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)
        
        # Forecast
        forecast_scaled = model.predict(X_test, verbose=0)
        forecast = scaler.inverse_transform(forecast_scaled)
        
        # Create forecast time series
        forecast_series = pd.Series(forecast.flatten(), 
                                    index=test_data.index[n_steps:])
        
        return forecast_series
         
    except Exception as e:
        print(f"\nError building LSTM model: {str(e)}")
        return None

# Perform LSTM forecast
lstm_pred = lstm_forecast(train['PM2.5'], test['PM2.5'])

# Visualize LSTM results
if lstm_pred is not None:
    plt.figure(figsize=(14, 6))
    plt.plot(train['PM2.5'], label='Training Data')
    plt.plot(test['PM2.5'], label='Actual Values')
    plt.plot(lstm_pred, label='LSTM Forecast', linestyle='--')
    plt.title('Forecasting using LSTM')
    plt.xlabel('Date')
    plt.ylabel('PM2.5 Concentration')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Forecast quality evaluation
def evaluate_forecast(actual, predicted, model_name):
    # Create DataFrame for easier processing
    eval_df = pd.DataFrame({
        'actual': actual,
        'predicted': predicted
    }).dropna()
    
    if len(eval_df) == 0:
        print(f"\n{model_name}: No data for evaluation")
        return
    
    # Calculate metrics
    mae = mean_absolute_error(eval_df['actual'], eval_df['predicted'])
    rmse = np.sqrt(mean_squared_error(eval_df['actual'], eval_df['predicted']))
    
    try:
        mape = np.mean(np.abs((eval_df['actual'] - eval_df['predicted']) / eval_df['actual'])) * 100
    except:
        mape = np.nan
    
    print(f"\nEvaluation results for {model_name}:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    if not np.isnan(mape):
        print(f"MAPE: {mape:.2f}%")
    print(f"Number of points for evaluation: {len(eval_df)}")

# Evaluate models
if arima_pred is not None:
    evaluate_forecast(test['PM2.5'], arima_pred, "ARIMA")

if hw_pred is not None:
    evaluate_forecast(test['PM2.5'], hw_pred, "Holt-Winters")

if lstm_pred is not None:
    evaluate_forecast(test['PM2.5'][len(test)-len(lstm_pred):], lstm_pred, "LSTM")

In [ ]:
# Conclusions and recommendations

print(" ARIMA usually performs well for stationary series")
print(" To improve results, try:")
print("   - Additional data cleaning")
print("   - Hyperparameter tuning for models")